# Processing data

En general los **datos** que tenemos para entrenar nuestro modelo no son la entrada directa a este y requieren ser transformados en **features** para obtener los mejores resultados.

$$data -> features -> modelo$$

Para lograr esto tenemos algunas opciones:
* hacer las transformaciones previas a entrenar el modelo y generar un dataset con las features
* incorporar estas transformaciones en el modelo construyendo un pipeline

En la primera opción podemos usar [pandas](https://pandas.pydata.org/) para procesar y transformar los datos generando el datset con todas las features y en la segunda opción podemos usar el modulo de [preprocessing de scikit-learn](https://scikit-learn.org/stable/modules/preprocessing.html) para aplicar estas transformaciones.

La ventaja de tener las transformaciones como una etapa dentro del modelo es que podemos aplicar transformación particulares a cada uno de nuestros modelos usando siempre el mismo dataset como entrada, mejorar la reproducibilidad y la mantención cuando los modelos están en producción.


La siguiente imagen es un esquema de cómo funcionan los transformer del módulo de preprocessing de scikit-learn
<img src="../images/transformer.png" alt="Scikit-learn Transformer" width=800/>


In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
from sklearn import feature_extraction
from sklearn import preprocessing
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

set_config(display='diagram')   

In [ ]:
train_path = '../data/users_train.csv'
train_data = pd.read_csv(train_path)

# Pandas

In [ ]:
x_numeric = train_data[['cnt_user_engagement']]

In [ ]:
mean = x_numeric.mean()
mean

In [ ]:
var = x_numeric.var()
var

In [ ]:
x_norm = (x_numeric - mean)/var**0.5
x_norm

# Scikit-learn
## Transformaciones Numéricas
**Lineales**
* `preprocessing.StandardScaler()`: transforma datos numéricos para que tengan media 0 y desviación estándar 1. $f(x) = \frac{x-\bar x}{\sigma}$
* `preprocessing.MinMaxScaler()`: transforma datos numéricos para que el mínimo valor sea 0 y el máximo sea 1

**No lineales**
* `preprocessing.QuantileTransformer()`: discretiza en quantiles una variable numérica y luego normaliza para obtener una distribución normal


In [ ]:
scaler = preprocessing.StandardScaler()

In [ ]:
scaler.fit(x_numeric)

In [ ]:
x_norm = scaler.transform(x_numeric)
x_norm

In [ ]:
scaler.mean_, scaler.var_

In [ ]:
f, ax = plt.subplots(2)
hist = ax[0].hist(x_norm)
hist = ax[1].hist(x_numeric)

>Prueba otras transformacion numerica con los mismos datos y revisa como es la salida de estas transformaciones

## Transformación categóricas 

* `preprocessing.OrdinalEncoder()`: transforma cada una de las categorías en un valor numerico, ej: cl, uy, in -> 0,1,2
* `preprocessing.OneHotEncoder()`: transforma cada una de las categorías en una columna con 0 y 1. ej: cl, uy, in -> [1,0,0], [0,1,0], [0,0,1]

En ambos casos la transformación no aceptan valores nulos por lo que debemos imputar los valores nulos antes de hacer esta.


### OrdinalEncoder

In [ ]:
x_categoric = train_data[['country_name']].dropna()

In [ ]:
ordinal_encoder = preprocessing.OrdinalEncoder()
ordinal_encoder.fit(x_categoric)

In [ ]:
ordinal_encoder.transform(x_categoric)

### One-Hot Encoder

In [ ]:
onehot_encoder = preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=False)
onehot_encoder.fit(x_categoric)

In [ ]:
onehot_encoder.transform(x_categoric)

In [ ]:
# si handle_unknown='error' cuando un valor este por fuera del vocabulario va a dar error
# onehot_encoder.transform([['xxxx']])

In [ ]:
# si handle_unknown='ignore' cuando un valor este por fuera del vocabulario la salida van a ser solo ceros
# onehot_encoder.transform([['xxxx']])

## Transformaciones en texto
* `feature_extraction.text.CountVectorizer`: es una transformacion similar a one-hot encoding pero con mas flexibilidad para procesar texto

In [ ]:
x_vector = x_categoric.iloc[:, 0].dropna().to_numpy()

In [ ]:
count_vectorize = feature_extraction.text.CountVectorizer(
    lowercase=True,
    tokenizer=lambda x:x.split('\n'),
    max_features=3,
    binary=True,
)

In [ ]:
count_vectorize.fit(x_vector)


In [ ]:
count_vectorize.transform(x_vector).todense()

In [ ]:
x_vector

In [ ]:
count_vectorize.vocabulary_

# Compose
## Pipeline
Scikit-learn nos permite construir un pipeline que no es más que una secuencia de pasos donde se definen transformaciones concatenadas que generan una salida única
<img src="../images/pipeline.png" alt="Scikit-learn Transformer" width=800/>


In [ ]:
transformer_numeric = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
        ("scaler", preprocessing.StandardScaler()),
    ]
)
transformer_numeric

In [ ]:
transformer_categorical = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("one-hot", preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=False)),
    ]
)
transformer_categorical

In [ ]:
CATEGORICAL_COLUMNS = [
    "country_name",
    "device_os",
    "device_lang",
]
NUMERICAL_COLUMNS = [
    "cnt_user_engagement",
    "cnt_level_start_quickplay",
    "cnt_level_end_quickplay",
    "cnt_level_complete_quickplay",
    "cnt_level_reset_quickplay",
    "cnt_post_score",
    "cnt_spend_virtual_currency",
    "cnt_ad_reward",
    "cnt_challenge_a_friend",
    "cnt_completed_5_levels",
    "cnt_use_extra_steps",
]
IGNORE_COLUMNS = [
    "user_first_engagement",
    "user_pseudo_id",
    "is_enable",
    "bounced",
    "device_lang",
]

In [ ]:
transformer_pipeline = ColumnTransformer(
    transformers=[
        ("numeric_features", transformer_numeric, NUMERICAL_COLUMNS),
        ("categorical_features", transformer_categorical, CATEGORICAL_COLUMNS),
        ("ignore_features", "drop", IGNORE_COLUMNS),
    ]
)
transformer_pipeline

In [ ]:
transformer_pipeline.fit(train_data)

In [ ]:
transformer_pipeline.transform(train_data)

In [ ]:
model_path = 'models'
if not os.path.exists(model_path):
    os.makedirs(model_path)

joblib.dump(transformer_pipeline, f'{model_path}/transformer_full.joblib')

## Pipeline más complejos

In [ ]:
transformer_numeric = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
        ("scaler", preprocessing.StandardScaler()),
    ]
)
transformer_numeric

In [ ]:
transformer_categorical = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        (
            "count_vectorize",
            ColumnTransformer(
                transformers=[
                    (
                        "feature_0",
                        feature_extraction.text.CountVectorizer(
                            lowercase=False,
                            tokenizer=lambda x: x.split("\n"),
                            max_features=3,
                            binary=True,
                        ),
                        0,
                    ),
                    (
                        "feature_1",
                        feature_extraction.text.CountVectorizer(
                            lowercase=False,
                            tokenizer=lambda x: x.split("\n"),
                            max_features=5,
                            binary=True,
                        ),
                        1,
                    ),
                    (
                        "feature_2",
                        feature_extraction.text.CountVectorizer(
                            lowercase=False,
                            tokenizer=lambda x: x.split("\n"),
                            max_features=8,
                            binary=True,
                        ),
                        2,
                    ),
                ]
            ),
        ),
    ]
)
transformer_categorical

In [ ]:
transformer_pipeline = ColumnTransformer(
    transformers=[
        ("numeric_features", transformer_numeric, NUMERICAL_COLUMNS),
        ("categorical_features", transformer_categorical, CATEGORICAL_COLUMNS),
        ("ignore_features", "drop", IGNORE_COLUMNS),
    ]
)
transformer_pipeline

In [ ]:
transformer_pipeline.fit_transform(train_data).shape